In [137]:
import pandas as pd 
data = pd.read_csv("../data/train.csv")
df = data.copy()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [138]:
df = df.drop(['PassengerId', 'Ticket','Cabin'], axis=1)

In [139]:
# Extraire le titre (le mot qui précède le point)
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')


In [140]:
df = df.drop('Name', axis=1)

In [141]:
age_median = df.groupby('Title')['Age'].transform('median')
print(age_median)
df['Age'] = df['Age'].fillna(age_median)

0      30.0
1      35.0
2      21.0
3      35.0
4      30.0
       ... 
886    48.5
887    21.0
888    21.0
889    30.0
890    30.0
Name: Age, Length: 891, dtype: float64


In [142]:
df['Age'].isna().sum()

np.int64(0)

In [143]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)


In [144]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone
0,0,3,male,22.0,1,0,7.2500,S,Mr,2,0
1,1,1,female,38.0,1,0,71.2833,C,Mrs,2,0
2,1,3,female,26.0,0,0,7.9250,S,Miss,1,1
3,1,1,female,35.0,1,0,53.1000,S,Mrs,2,0
4,0,3,male,35.0,0,0,8.0500,S,Mr,1,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Rare,1,1
887,1,1,female,19.0,0,0,30.0000,S,Miss,1,1
888,0,3,female,21.0,1,2,23.4500,S,Miss,4,0
889,1,1,male,26.0,0,0,30.0000,C,Mr,1,1


In [145]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace= True)

C:\Users\chris\AppData\Local\Temp\ipykernel_28032\4228943529.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace= True)


In [146]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
categorial_features = ['Sex', 'Embarked', 'Title','Pclass']
numeric_features = ['Age','FamilySize', 'IsAlone', 'Fare']


In [147]:
from sklearn.compose import ColumnTransformer
nominal_transform = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown= 'ignore'))
])

numeric_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transform, numeric_features),
        ('cat_nominal', nominal_transform, categorial_features)
    ]
)


In [148]:
df_final = df.drop(['SibSp', 'Parch'], axis=1)
df_final

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone
0,0,3,male,22.0,7.2500,S,Mr,2,0
1,1,1,female,38.0,71.2833,C,Mrs,2,0
2,1,3,female,26.0,7.9250,S,Miss,1,1
3,1,1,female,35.0,53.1000,S,Mrs,2,0
4,0,3,male,35.0,8.0500,S,Mr,1,1
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,13.0000,S,Rare,1,1
887,1,1,female,19.0,30.0000,S,Miss,1,1
888,0,3,female,21.0,23.4500,S,Miss,4,0
889,1,1,male,26.0,30.0000,C,Mr,1,1


In [149]:
x = df_final.drop('Survived', axis= 1)
y = df_final['Survived']

In [150]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0)

In [151]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Liste de modèles à tester
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42)
}

# Comparer
for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    scores = cross_val_score(pipeline, x_train, y_train, cv=5, scoring='accuracy')
    print(f"{name}: {scores.mean():.3f} (+/- {scores.std():.3f})")

Logistic Regression: 0.822 (+/- 0.028)
Random Forest: 0.793 (+/- 0.033)
SVM: 0.829 (+/- 0.023)


In [152]:

best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])


best_pipeline.fit(x_train, y_train)


val_score = best_pipeline.score(x_val, y_val)


# Comparaison
print(f"CV Score:         {0.829:.3f}")
print(f"Validation Score: {val_score:.3f}")


CV Score:         0.829
Validation Score: 0.821


Notre modèle généralise donc bien 

In [153]:
best_pipeline.fit(x,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'FamilySize',
                                                   'IsAlone', 'Fare']),
                                                 ('cat_nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked', 'Title',
                                                   'Pclass'])])),
                ('classifier', SVC(random_state=42))])

In [154]:

def preprocess_data(df):
    df = df.copy()
    
    # 1. Extraire Title
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_mapping = {
        'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
        'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare',
        'Col': 'Rare', 'Don': 'Rare', 'Dr': 'Rare',
        'Major': 'Rare', 'Rev': 'Rare', 'Sir': 'Rare',
        'Jonkheer': 'Rare', 'Dona': 'Rare'
    }
    df['Title'] = df['Title'].replace(title_mapping)
    
    # 2. Imputer Age par Title
    age_median = df.groupby('Title')['Age'].transform('median')
    df['Age'] = df['Age'].fillna(age_median)
    
    # 3. FamilySize et IsAlone
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # 4. Imputer Embarked
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    
    # 5. Supprimer colonnes inutiles
    df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], 
                  axis=1, errors='ignore')
    
    return df


In [155]:
test_data = pd.read_csv("../data/test.csv")
df_test = preprocess_data(test_data)


C:\Users\chris\AppData\Local\Temp\ipykernel_28032\2159945985.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


In [158]:
predictions = best_pipeline.predict(df_test)

In [160]:
df_test['Prédictions'] = predictions
df_test

,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone,Prédictions
0,3,male,34.5,7.8292,Q,Mr,1,1,0
1,3,female,47.0,7.0000,S,Mrs,2,0,0
2,2,male,62.0,9.6875,Q,Mr,1,1,0
3,3,male,27.0,8.6625,S,Mr,1,1,0
4,3,female,22.0,12.2875,S,Mrs,3,0,1
...,...,...,...,...,...,...,...,...,...
413,3,male,28.5,8.0500,S,Mr,1,1,0
414,1,female,39.0,108.9000,C,Rare,1,1,1
415,3,male,38.5,7.2500,S,Mr,1,1,0
416,3,male,28.5,8.0500,S,Mr,1,1,0


In [164]:
# 1. Nombre de prédictions
print(f"Nombre de prédictions : {len(predictions)}")
print(f"Nombre attendu (test set) : {len(df_test)}")

# 2. Distribution des prédictions
unique, counts = np.unique(predictions, return_counts=True)
print(f"\nDistribution des prédictions :")
for val, count in zip(unique, counts):
    label = "Morts" if val == 0 else "Survivants"
    print(f"  {label} (classe {val}) : {count} ({count/len(predictions)*100:.1f}%)")

# 3. Comparer avec la distribution du train
print(f"\nDistribution du train set :")
print(y.value_counts(normalize=True))

Nombre de prédictions : 418
Nombre attendu (test set) : 418

Distribution des prédictions :
  Morts (classe 0) : 263 (62.9%)
  Survivants (classe 1) : 155 (37.1%)

Distribution du train set :
Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64
